In [1]:
import findspark 
findspark.init()

In [2]:
from pyspark.sql import SparkSession
import shutil
import glob

from configparser import ConfigParser
import mysql.connector
import pandas as pd

In [3]:
spark = SparkSession \
    .builder \
    .appName("Data_Pipeline") \
    .getOrCreate()

sc = spark.sparkContext

print("session démarrée, son id est ", sc.applicationId)

session démarrée, son id est  local-1685284060978


In [14]:
config = ConfigParser()
config.read('./Config.ini')

['./Config.ini']

In [25]:
url = config.get('mysql', 'host')
user = config.get('mysql', 'user')
pwd  = config.get('mysql', 'password')
db =  config.get('mysql', 'database')
pt = config.get('mysql', 'port')

In [42]:
# Etablir une connexion à MySQL
conn = mysql.connector.connect(user= user, database=db,
                               password=pwd,
                               host=url,
                               port=pt)
cursor = conn.cursor()
query = "select * from data"

# Execution de la requête et sauvegarde du résultat dans un dataframe pandas
pdf = pd.read_sql(query, con=conn)
conn.close()

C:\Users\mouha\AppData\Local\Temp\ipykernel_19764\3200221813.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pdf = pd.read_sql(query, con=conn)


In [43]:
pdf

,id,city,temperature,humidity,windSpeed,timestamp
0,1,Dakar,84,54,162,2023-05-28 14:44:43
1,2,Dakar,29,65,8,2023-05-28 16:59:44
2,3,Dakar,29,65,8,2023-05-28 16:59:44
3,4,Dakar,28,69,9,2023-05-28 18:06:12


In [22]:
APIKEY = config.get("openweathermap", "APIKEY")
URL = config.get("openweathermap", "URL")

city = {
  "name": "Dakar",
  "lat": 14.67,
  "lon": -17.44
}

In [40]:
import requests
import json
from datetime import datetime
# url = "https://api.open-meteo.com/v1/forecast?latitude=14.69&longitude=-17.44&hourly=temperature_2m,relativehumidity_2m,windspeed_10m"
api_url = f"{URL}?lat={city['lat']}&lon={city['lon']}&appid={APIKEY}&units=metric"

data = requests.get(api_url)
data_parsed = json.loads(data.text)
print(data_parsed)
data_formatted = {
  "city": city['name'],
  "temperature": data_parsed['main']['temp'],
  "humidity": data_parsed['main']['humidity'],
  "windSpeed": data_parsed['wind']['speed'],
  "timestamp": datetime.utcfromtimestamp(data_parsed['dt']).strftime('%Y-%m-%d %H:%M:%S'),
}

{'coord': {'lon': -17.44, 'lat': 14.67}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 27.99, 'feels_like': 30.52, 'temp_min': 27.99, 'temp_max': 32.42, 'pressure': 1008, 'humidity': 69}, 'visibility': 10000, 'wind': {'speed': 8.75, 'deg': 360}, 'clouds': {'all': 0}, 'dt': 1685297172, 'sys': {'type': 1, 'id': 2419, 'country': 'SN', 'sunrise': 1685255993, 'sunset': 1685302467}, 'timezone': 0, 'id': 2595757, 'name': 'Central Dakar', 'cod': 200}


In [38]:
from pyspark.sql.types import *

# Puisque nous connaissons déjà le format des données, définissons le schéma pour accélérer le traitement (pas besoin de Spark pour déduire le schéma)
jsonSchema = StructType()\
    .add("id", StringType())\
    .add("city", StringType(), True)\
    .add("temperature", StringType(), True)\
    .add("humidity", StringType(), True)\
    .add("windSpeed", StringType(), True)\
    .add("timestamp", StringType())\
# DataFrame statique représentant les données dans les fichiers JSON
staticInputDF = (
  spark
    .read
    .schema(jsonSchema)
    .json(df.rdd)
)

staticInputDF.show()

+----+----+-----------+--------+---------+---------+
|  id|city|temperature|humidity|windSpeed|timestamp|
+----+----+-----------+--------+---------+---------+
|null|null|       null|    null|     null|     null|
|null|null|       null|    null|     null|     null|
|null|null|       null|    null|     null|     null|
+----+----+-----------+--------+---------+---------+



In [41]:
try:
  conn = mysql.connector.connect(user= user, database=db, password=pwd, host=url, port=pt)
  cursor = conn.cursor()
  query = f"""
  INSERT INTO `data` (`city`, `temperature`, `humidity`, `windSpeed`, `timestamp`) 
  VALUES (
    '{data_formatted['city']}', 
    '{data_formatted['temperature']}', 
    '{data_formatted['humidity']}', 
    '{data_formatted['windSpeed']}', 
    '{data_formatted['timestamp']}'
  )"""

  cursor.execute(query)
  conn.commit()
  print(cursor.rowcount, "Record inserted successfully into data table")
  conn.close()
except mysql.connector.Error as error:
  print("Failed to insert record into Laptop table {}".format(error))

finally:
  if conn.is_connected():
    conn.close()
    print("MySQL connection is closed")

1 Record inserted successfully into data table


In [122]:
import requests
import json
from pyspark.sql.functions import udf, col, explode
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType

In [123]:
from pyspark.sql import Row
headers = {
  'content-type': "application/json"
}
body = json.dumps({})
RestApiRequestRow = Row("verb", "url", "headers", "body")

api_url = "https://api.openweathermap.org/data/2.5/onecall?lat=14.497401&lon=-14.452362&exclude=hourly,daily&appid=101098fb7b42c64a657c60649632e063"
request_df = spark.createDataFrame([
  RestApiRequestRow("get", api_url, headers, body)
])

request_df.show()

+----+--------------------+--------------------+----+
|verb|                 url|             headers|body|
+----+--------------------+--------------------+----+
| get|https://api.openw...|{content-type -> ...|  {}|
+----+--------------------+--------------------+----+



In [124]:
schema = StructType([
    StructField("lat", DoubleType(), True),
    StructField("lon", DoubleType(), True),
    StructField("timezone", StringType(), True),
    StructField("timezone_offset", IntegerType(), True),
    StructField("current", StructType([
        StructField("dt", LongType(), True),
        StructField("sunrise", LongType(), True),
        StructField("sunset", LongType(), True),
        StructField("temp", DoubleType(), True),
        StructField("feels_like", DoubleType(), True),
        StructField("pressure", IntegerType(), True),
        StructField("humidity", IntegerType(), True),
        StructField("dew_point", DoubleType(), True),
        StructField("uvi", IntegerType(), True),
        StructField("clouds", IntegerType(), True),
        StructField("visibility", IntegerType(), True),
        StructField("wind_speed", DoubleType(), True),
        StructField("wind_deg", IntegerType(), True),
        StructField("wind_gust", DoubleType(), True),
        StructField("weather", ArrayType(StructType([
            StructField("id", IntegerType(), True),
            StructField("main", StringType(), True),
            StructField("description", StringType(), True),
            StructField("icon", StringType(), True)
        ])), True)
    ]), True)
])

In [125]:
def executeRestApi(verb, url, headers, body):
  #
  headers = {
      'content-type': "application/json"
  }
  res = None
  # Make API request, get response object back, create dataframe from above schema.
  try:
    if verb == "get":
      res = requests.get(url, data=body, headers=headers)
    else:
      res = requests.post(url, data=body, headers=headers)
  except Exception as e:
    return e
  if res != None and res.status_code == 200:
    return json.loads(res.text)
  return None

In [57]:
udf_executeRestApi = udf(executeRestApi, schema)
udf_executeRestApi

<function __main__.executeRestApi(verb, url, headers, body)>

In [100]:
udf_executeRestApi(col("verb"), col("url"), col("headers"), col("body"))

Column<'executeRestApi(verb, url, headers, body)'>

In [126]:
result_df = request_df \
             .withColumn("data", udf_executeRestApi(col("verb"), col("url"), col("headers"), col("body")))

In [67]:
from pyspark.sql import SparkSession
# from pyspark.dbutils import DBUtils
from pyspark.sql.functions import col
from pyspark.sql.types import StructType

spark = SparkSession.builder.getOrCreate()

# declare dummy data to demonstrate how the collapse mechanism works
jsonStrings = ['{"car":{"color":"red", "model":"jaguar"},"name":"Jo","address":{"city":"Houston","state":"Texas", "zip":{"first":1234,"second":4321}}}']
otherPeopleRDD = spark.sparkContext.parallelize(jsonStrings)
df = spark.read.json(otherPeopleRDD)
  
# Recursively iterates over the schema, creating an array of arrays, whereby each item
# of the master array, is an array of column names
#
# For example, lets say there are three columns of which two are hierarchical and the following schema/structure
#    name
#    address
#      street
#      town
#    details
#      age
#      gender
#
# The function will return the following array:
# [["name"],["address","street"],["address","town"],["details","age"],["details","gender"]]
def get_all_columns_from_schema(source_schema):
  branches = []
  def inner_get(schema, ancestor=None):
    if ancestor is None: ancestor = []
    for field in schema.fields:
      branch_path = ancestor+[field.name]     
      if isinstance(field.dataType, StructType):    
        inner_get(field.dataType, branch_path) 
      else:
        branches.append(branch_path)
        
  inner_get(source_schema)
        
  return branches

# collapse_columns is passed the dataframe schema, which is then passes
# to get_all_columns_from_schema.  On return, it iterates through the array
# of columns in order to build up the select list that will be used
# to collapse the hierarchical columns into a single 2d structure
#
# for example, lets say _all_columns has the following array: [["name"],["address","street"]]
# after iterating through the array, the function response will be
# [col("name"), col("address.street").alias("address_street")]
def collapse_columns(source_schema, columnFilter=None):
  _columns_to_select = []
  if columnFilter is None: columnFilter = ""
  _all_columns = get_all_columns_from_schema(source_schema)
  for column_collection in _all_columns:
    if (len(columnFilter) > 0) & (column_collection[0] != columnFilter): 
        continue

    # columns with questionable character choices like a space, need to be wrapped
    # in `` characters.  The alias function will do this automatically, but the selection of the column
    # e.g. col("col name") will not
    select_column_collection = ['`%s`' % list_item for list_item in column_collection]    
    
    if len(column_collection) > 1:
      _columns_to_select.append(col('.'.join(select_column_collection)).alias('_'.join(column_collection)))
    else:
      _columns_to_select.append(col(select_column_collection[0]))

  return _columns_to_select

# as above but for individual columns
def collapse_column(source_df, source_column):
    column_name = ""
    if isinstance(source_column, Column):
      column_name = source_column.name
    else:
      column_name = source_column

    return collapse_columns(source_df.schema, column_name)

# returns a dataframe that has been collapsed.  Input is the dataframe to be collapsed
def collapse_to_dataframe(source_df):
  return source_df.select(collapse_columns(source_df.schema))
  
# now test
collapse_to_dataframe(df).show()

+------------+-------------+-----------------+------------------+---------+---------+----+
|address_city|address_state|address_zip_first|address_zip_second|car_color|car_model|name|
+------------+-------------+-----------------+------------------+---------+---------+----+
|     Houston|        Texas|             1234|              4321|      red|   jaguar|  Jo|
+------------+-------------+-----------------+------------------+---------+---------+----+



In [74]:
# df = result_df.select(explode(col("result.current")).alias("results"))
# df.select(collapse_columns(df.schema)).show()

In [128]:
result_df.show()

+----+--------------------+--------------------+----+--------------------+
|verb|                 url|             headers|body|                data|
+----+--------------------+--------------------+----+--------------------+
| get|https://api.openw...|{content-type -> ...|  {}|{14.4974, -14.452...|
+----+--------------------+--------------------+----+--------------------+



In [110]:
# result_df.select('data').write.csv("./data")

AnalysisException: Column `data` has a data type of struct<lat:double,lon:double,timezone:string,timezone_offset:int,current:struct<dt:bigint,sunrise:bigint,sunset:bigint,temp:double,feels_like:double,pressure:int,humidity:int,dew_point:double,uvi:int,clouds:int,visibility:int,wind_speed:double,wind_deg:int,wind_gust:double,weather:array<struct<id:int,main:string,description:string,icon:string>>>>, which is not supported by CSV.

In [130]:
result_select_df = result_df.select('data')

In [139]:
# result_select_df.
# df=spark.createDataFrame(result_select_df,schema)

# df.show(5,False)
import json

print(json.loads(result_select_df.toJSON()) )

TypeError: the JSON object must be str, bytes or bytearray, not RDD

In [84]:
result_df.select('result').show()

+--------------------+
|              result|
+--------------------+
|{14.4974, -14.452...|
+--------------------+



In [ ]:
result_select_df

In [99]:
import struct
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType

# schema = StructType(
#     [
#         StructField('lat', StringType(), True),
#         StructField('lon', StringType(), True)
#     ]
# )

result_df.withColumn("result", struct("result", schema))\
    .select(col('result.*'))\
    .show()

AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "from_json(result)" due to data type mismatch: Parameter 1 requires the "STRING" type, however "result" has the type "STRUCT<lat: DOUBLE, lon: DOUBLE, timezone: STRING, timezone_offset: INT, current: STRUCT<dt: BIGINT, sunrise: BIGINT, sunset: BIGINT, temp: DOUBLE, feels_like: DOUBLE, pressure: INT, humidity: INT, dew_point: DOUBLE, uvi: INT, clouds: INT, visibility: INT, wind_speed: DOUBLE, wind_deg: INT, wind_gust: DOUBLE, weather: ARRAY<STRUCT<id: INT, main: STRING, description: STRING, icon: STRING>>>>".;
'Project [verb#227, url#228, headers#229, body#230, from_json(StructField(lat,DoubleType,true), StructField(lon,DoubleType,true), StructField(timezone,StringType,true), StructField(timezone_offset,IntegerType,true), StructField(current,StructType(StructField(dt,LongType,true),StructField(sunrise,LongType,true),StructField(sunset,LongType,true),StructField(temp,DoubleType,true),StructField(feels_like,Double